In [ ]:
import json
import os
from time import sleep
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd

def fetch_crypto_data():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start': '1',
        'limit': '15',
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': 'e4a3bb1a-739c-4dd7-a6d9-651d3033e436',
    }
    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        crypto_data = json.loads(response.text)
        return crypto_data
    except (ConnectionError, Timeout, TooManyRedirects) as error:
        print(error)
        return None

def normalize_crypto_data(data):
    crypto_df = pd.json_normalize(data['data'])
    crypto_df['timestamp'] = pd.to_datetime('now')
    return crypto_df

def api_runner(existing_data=None):
    crypto_data = fetch_crypto_data()
    if crypto_data is None:
        return existing_data
    
    normalized_data = normalize_crypto_data(crypto_data)
    
    if existing_data is None:
        return normalized_data
    else:
        return pd.concat([existing_data, normalized_data], ignore_index=True)

def collect_and_save_data(iterations=20, filepath='/home/rafael/Documents/Python/CryptoAPI/crypto_data.csv'):
    collected_data = pd.DataFrame()
    
    for _ in range(iterations):
        collected_data = api_runner(collected_data)
        print('API Runner completed successfully')
        
        if not os.path.isfile(filepath):
            collected_data.to_csv(filepath, index=False)
        else:
            collected_data.to_csv(filepath, mode='a', header=False, index=False)
        
        sleep(86400)  # Sleep for 24 hours (86400 seconds)

In [ ]:
if __name__ == "__main__":
    collect_and_save_data()